In [237]:
import pandas as pd
import os

In [238]:
xls = pd.ExcelFile('504_Spark_Project_Part 1.xlsx')

In [239]:
role = ['President','Senate','Governor','Lieutenant_Governor','Treasurer','Auditor']

In [240]:
for each in role:
    globals()['score_{}'.format(each)] = pd.read_excel(xls, each)

In [241]:
csv_name = list(os.walk('/Users/chuci/Desktop/cs504/spark/vote'))[0][2]
df_name =[]
for each in csv_name:
    name = each.split('_')
    name = '{}_{}_{}'.format(name[3],name[4],name[2])
    df_name.append(name)
    globals()[name] = pd.read_csv('vote/{}'.format(each))

In [243]:
for each in df_name:
    col_name = globals()[each].columns
    modify_name = []
    for name in col_name:
        if '/' in name:
            modify_name.append(name)
    dic = {}
    for name in modify_name:
        dic[name] = name.replace('/',' and')
    globals()[each].rename(columns=dic, inplace = True)
    globals()[each].drop(globals()[each].shape[0]-1,inplace = True)

In [244]:
def get_score(x):
    try:
        return ((x[name1])*(score_candidate1) + (x[name2])*(score_candidate2))/((x[name1]) + (x[name2]))
    except:
        return 0

In [269]:
for each in df_name:
    job,stage,year = each.split('_')
    col = globals()[each].columns
    name1 = col[3]
    name2 = col[4]
    area_score = globals()[each]
    area_score['new_index'] = area_score['City andTown'] + area_score['Ward'] +  area_score['Pct']
    area_score[name1].replace(',','',regex = True,inplace = True)
    area_score[name1] = globals()[each][name1].astype('double')
    area_score[name2].replace(',','',regex = True,inplace = True)
    area_score[name2] = globals()[each][name1].astype('double')
    score_table = globals()['score_{}'.format(job)]
    score_candidate1 = score_table[(score_table['Name'] == name1) & (score_table['Year'] == int(year)) & (score_table['Type of Election'] == stage)]['Level of Progressive/Conservative'].iloc[0]
    score_candidate2 = score_table[(score_table['Name'] == name2) & (score_table['Year'] == int(year)) & (score_table['Type of Election'] == stage)]['Level of Progressive/Conservative'].iloc[0]
    area_score['area_score'] = area_score.apply(get_score,axis = 1 )
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [261]:
catage = {}
for name in role:
    for each in df_name:
        if name in each:
            if name in catage:
                catage[name].append(each)
            else:
                catage[name] = [each]

In [408]:
for each in role:
    middle = globals()[df_name[0]][['Ward','new_index']]
    count = 0
    try:
        for item in catage[each]:
            middle = middle.merge(globals()[item],on = 'new_index')
            middle.rename(columns={'area_score_x':'area_score_{}'.format(str(count)),'area_score_y':'area_score_{}'.format(str(count+1))},inplace  = True)
            count += 1
        middle = middle[['new_index'] + list(set(middle[[x for x in middle.columns if 'area_score' in x]]))]
        middle['fina_score'] = middle.mean(1)
        globals()['{}_final_score'.format(each)] = middle
    except:
        continue

In [413]:
for each in role:
    name = '{}_final_score'.format(each)
    try:
        globals()[name].to_csv(name+'.csv',index = False)
    except:
        continue